# Implementation of "Procedural Fairness Through Decoupling Objectionable Data Generating Components"

Experiment on [UCI Adult data set](https://archive.ics.uci.edu/dataset/2/adult).

Following Nabi \& Shpitser (2018) and Chiappa (2019), the potential locations for objectionable components include:
- $A \rightarrow Y$
- $A \rightarrow M \rightarrow \cdots \rightarrow Y$
  - $A \rightarrow M \rightarrow Y$
  - $A \rightarrow M \rightarrow L \rightarrow Y$
  - $A \rightarrow M \rightarrow R \rightarrow Y$
  - $A \rightarrow M \rightarrow L \rightarrow R \rightarrow Y$

1. We define the causal graph `CausalGraph` by specifying the list of nodes and their parents.
1. We construct the prediction model `CustomNetwork` using `SubNetworks` for each local causal module, i.e., for nodes that have parent nodes in the `CausalGraph`.
1. We implement training and testing afterwards, followed by the optimization for reference points.

## Train the nonlinear model without fairness constraints

In [ ]:
# Python package for simulated annealing
%pip install -e git+https://github.com/perrygeo/simanneal.git#egg=simanneal

### Define causal model and load data

In [ ]:
# Import libraries
import os
import numpy as np
from pprint import pprint
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset, SubsetRandomSampler
from utils import (
    CustomDataset,
    CausalGraph,
    CustomMinMaxScaler,
    CustomNetwork,
    CustomStandardScaler,
)
from utils import (
    train_hierarchical,
    evaluate_hierarchical,
    load_uci_adult_preprocessed,
    TailRefPtConfigAnnealer,
)

if_train = True

In [ ]:
# Define causal graph and prepare data
experiment_prefix = "uci_adult"
relation_type = "nonlinear"
experiment_prefix = experiment_prefix + "." + relation_type
is_linear = True if "linear" == relation_type else False

nodes = [
    "male",  # sex
    "married",  # marital-status
    "higher_edu",  # education-num
    "managerial_occ",  # occupation
    "high_hours",  # hours-per-week
    "gov_jobs",  # work-class
    # "race",  # not included in prediction process
    "age",
    "native_country",  # native-country
    "high_income",  # income
]
parent_dict = {
    "male": [],  # A
    "age": [],  # C (part)
    "native_country": [],  # C (part)
    "married": [
        "male",
        "age",
        "native_country",
    ],  # M
    "higher_edu": [
        "male",
        "age",
        "native_country",
        "married",
    ],  # L
    "managerial_occ": [
        "male",
        "age",
        "native_country",
        "married",
        "higher_edu",
    ],  # R1
    "high_hours": [
        "male",
        "age",
        "native_country",
        "married",
        "higher_edu",
    ],  # R2
    "gov_jobs": [
        "male",
        "age",
        "native_country",
        "married",
        "higher_edu",
    ],  # R3
    "high_income": [
        "male",
        "age",
        "native_country",
        "married",
        "higher_edu",
        "managerial_occ",
        "high_hours",
        "gov_jobs",
    ],  # Y
}
node_types = {node: "continuous" for node in nodes}
node_types["male"] = "binary"
node_types["high_income"] = "binary"
causal_graph = CausalGraph(nodes, parent_dict, node_types)
final_output_node = causal_graph.nodes[-1]

data_dict, n_samples = load_uci_adult_preprocessed(
    nodes, data_file_path="data/adult_processed.csv"
)
print(f"Number of samples: {n_samples}.")

### Experiment preparation

In [ ]:
# Specify experimental setting
learning_rate = 1e-3
min_hidden_neurons = 0 if "linear" == relation_type else 5
if 0 != min_hidden_neurons:
    experiment_prefix += f"_min_hidden_neurons_{min_hidden_neurons}"

if_linear_model = is_linear

# GPU index (if use cuda)
gpu_idx = 0

n_epochs = 100
batch_size = 2048
checkpoint_interval = 10
n_repeats = 1 if if_train is True else 0  # number of runs

# Used only in linear models, will be ignored for nonlinear models
lambda_linear_L1 = 0.0
# No explicit edge coef constraint in our approach
edge_linear_coef_constraint_config = None  # set to {...} for linear

# Set device
device_type = "cuda" if torch.cuda.is_available() else "cpu"
device_id = torch.device(f"cuda:{gpu_idx}") if "cuda" == device_type else "cpu"
device = torch.device(device_id)
experiment_prefix = experiment_prefix + "." + device_type
print(f"Device: {device}")

# Set checkpoint directory and file name
checkpoint_dir = os.path.join(os.getcwd(), "checkpoints")
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_file = os.path.join(checkpoint_dir, f"{experiment_prefix}.pt")

print(f"Experiment prefix: {experiment_prefix}")

### Data set processing

- For features: shift and scale to 0 mean and 1 variance
- For targets: shift and scale to [0, 1] without clipping

In [ ]:
# Data preprocessing and data set preparation
# Split the indices into train and test indices (fix testing indices)
train_indices, test_indices = train_test_split(
    np.arange(n_samples), test_size=0.25, random_state=42
)

# Create StandardScaler for features and fit on train_indices (later into train_ and val_indices)
feature_standard_scaler_dict = CustomStandardScaler()
feature_standard_scaler_dict.fit(
    data_dict,
    [  # excluded nodes that are not features
        final_output_node,
    ],
    train_indices,
)

# Create MinMaxScaler and scale final output to [0, 1] on train_indices
target_minmax_scaler_dict = CustomMinMaxScaler((0, 1), clip=False)
target_minmax_scaler_dict.fit(
    data_dict,
    [  # only include target nodes
        final_output_node,
    ],
    train_indices,
)

# Transform features and targets sequentially
# NOTE use args instead of kwargs to avoid `wrapped()` error of sklearn
_training_data_dict_scaled = feature_standard_scaler_dict.transform(
    data_dict,
    [  # excluded keys
        final_output_node,
    ],
    train_indices,
)
training_data_dict_scaled = target_minmax_scaler_dict.transform(
    _training_data_dict_scaled,
    [  # target keys
        final_output_node,
    ],
    np.arange(len(train_indices)),
)

_testing_data_dict_scaled = feature_standard_scaler_dict.transform(
    data_dict,
    [  # excluded keys
        final_output_node,
    ],
    test_indices,
)
testing_data_dict_scaled = target_minmax_scaler_dict.transform(
    _testing_data_dict_scaled,
    [  # target keys
        final_output_node,
    ],
    np.arange(len(test_indices)),
)

# Create CustomDataset instances
training_dataset = CustomDataset(training_data_dict_scaled)
testing_dataset = CustomDataset(testing_data_dict_scaled)

### Train the neural network with cross validation

Indicative running time: ~ 2 min

If the current experimental setting is already trained previously, skip this section and directly load the trained model.

In [ ]:
# Train the neural network with cross validation
best_checkpoint_val_loss = float("inf")

for _ in range(n_repeats):
    # Split training_dataset into training, validating
    _train_indices, _val_indices = train_test_split(
        np.arange(len(training_dataset)), test_size=0.25
    )

    # Create DataLoader on training_dataset
    train_loader = DataLoader(
        training_dataset,
        batch_size=batch_size,
        sampler=SubsetRandomSampler(_train_indices),
    )
    val_loader = DataLoader(
        training_dataset,
        batch_size=batch_size,
        sampler=SubsetRandomSampler(_val_indices),
    )

    # Initialize model, criterion, and optimizer
    model = CustomNetwork(
        causal_graph, min_hidden_neurons=min_hidden_neurons, is_linear=if_linear_model
    ).to(device)

    criterion_dict = {
        node: nn.BCELoss() if submodel.output_type == "binary" else nn.MSELoss()
        for node, submodel in model.submodels.items()
    }

    optimizer_dict = {
        node: torch.optim.Adam(submodel.parameters(), lr=learning_rate)
        for node, submodel in model.submodels.items()
    }

    model.set_lambda_linear_model_coef_penalty_L1(lambda_linear_L1)

    for epoch in range(n_epochs):
        train_loss_dict = train_hierarchical(
            model,
            train_loader,
            criterion_dict,
            optimizer_dict,
            edge_linear_coef_constraint_config,
        )
        val_loss_dict = evaluate_hierarchical(
            model, val_loader, criterion_dict, feature_standard_scaler_dict
        )

        # Save the top-1 best performance checkpoints
        if epoch % checkpoint_interval == 0:
            # Sum over all output nodes of local modules
            val_loss = sum(val_loss_dict.values())
            if val_loss < best_checkpoint_val_loss:
                best_checkpoint_val_loss = val_loss

                # Save the checkpoint
                torch.save(
                    {
                        "epoch": epoch,
                        "model_state_dict": model.state_dict(),
                        # 'optimizer_state_dict': optimizer.state_dict(),
                        # 'loss': val_loss,
                        "hyperparams": {
                            "learning_rate": learning_rate,
                            "min_hidden_neurons": min_hidden_neurons,
                        },
                    },
                    checkpoint_file,
                )

pprint(f"Best model state (checkpoint_file): {checkpoint_file}")

## Derive $\widehat{Y}$ with reference point

For better interpretability, the reference point configuration is always in the raw data scale.
The corresponding scaling for prediction is included in `CustomNetwork`.

In this implementation, disadvantaged individuals are female individuals, but there are different reference point configurations:
1. $\mathcal{E}_{\mathrm{obj}} = \{ A \rightarrow Y \}$
1. $\mathcal{E}_{\mathrm{obj}} = \{ A \rightarrow Y, M \rightarrow Y \}$
1. $\mathcal{E}_{\mathrm{obj}} = \{ A \rightarrow Y, M \rightarrow R_1, M \rightarrow Y \}$
1. $\mathcal{E}_{\mathrm{obj}} = \{ A \rightarrow Y, M \rightarrow R_1, M \rightarrow L, M \rightarrow Y \}$

Note:
- There is no fairness constraint enforced when fitting model parameters
- The reference points are introduced to decouple objectionable components

### Load the trained model

In [ ]:
# Load the model
def load_best_model(checkpoint_dir, experiment_prefix):
    files = os.listdir(checkpoint_dir)
    matching_files = [f for f in files if f.startswith(experiment_prefix)]
    # matching_files.sort(key=lambda x: int(x.split('.')[1].split('_')[1]))
    best_checkpoint_file = matching_files[0]
    print(best_checkpoint_file)
    checkpoint = torch.load(os.path.join(checkpoint_dir, best_checkpoint_file))
    model = CustomNetwork(
        causal_graph,
        min_hidden_neurons=checkpoint["hyperparams"]["min_hidden_neurons"],
        is_linear=if_linear_model,
    ).to(device)
    model.load_state_dict(checkpoint["model_state_dict"])
    return model, checkpoint["hyperparams"]


best_model, best_hyperparams = load_best_model(checkpoint_dir, experiment_prefix)

criterion_dict = {
    node: nn.BCELoss() if submodel.output_type == "binary" else nn.MSELoss()
    for node, submodel in best_model.submodels.items()
}

### Define most disadvantaged individuals

In [ ]:
# DataLoader for data points where REFERENCE_POINT need to be set
mask_most_disadv_individuals = training_dataset.torch_tensor_dict["male"] < 0
mask_length = mask_most_disadv_individuals.shape[0]
idx_most_disadv_individuals = np.arange(mask_length)[
    mask_most_disadv_individuals.reshape(
        -1,
    )
]
most_disadv_individuals = Subset(training_dataset, idx_most_disadv_individuals)

record_loader = DataLoader(
    most_disadv_individuals, batch_size=len(most_disadv_individuals)
)  # only one batch

### Perform simulated annealing to derive the best reference point values

#### Simulated annealing configuration

In [ ]:
# Find best reference points by Simulated Annealing
# Create MinMaxScaler for features are in its raw scale
feature_minmax_scaler_dict = CustomMinMaxScaler((0, 1), clip=False)
feature_nodes = nodes.copy()
feature_nodes.remove(final_output_node)
feature_minmax_scaler_dict.fit(
    data_dict, feature_nodes, np.arange(len(train_indices))  # included nodes
)

# Different number of tail reference points initialization with raw-scale variable values

initial_tail_ref_pt_config = {
    "male->high_income": 1,
    # "married->higher_edu": 1,
    # "married->managerial_occ": 1,
    "married->high_income": 1,
}

best_overall_tail_ref_pt_config = None
best_overall_avg_outcome = float("-inf")

# Create an instance of CustomNetworkAnnealer
annealer = TailRefPtConfigAnnealer(
    state=initial_tail_ref_pt_config,
    model=best_model,
    dataloader=record_loader,
    node_types=node_types,
    output_node=final_output_node,
    feature_standard_scaler_dict=feature_standard_scaler_dict,
    feature_minmax_scaler_dict=feature_minmax_scaler_dict,
)

#### Multiple run to find the best reference point values

Indicative running time: ~ 1 min per repeat

In [ ]:
# Number of times to run the simulated annealing process
n_repeats_SA = 1

for i in range(n_repeats_SA):
    # Create an instance of CustomNetworkAnnealer
    annealer = TailRefPtConfigAnnealer(
        state=initial_tail_ref_pt_config,
        model=best_model,
        dataloader=record_loader,
        node_types=node_types,
        output_node=final_output_node,
        feature_standard_scaler_dict=feature_standard_scaler_dict,
        feature_minmax_scaler_dict=feature_minmax_scaler_dict,
    )

    annealer.set_schedule({"tmax": 5.0, "tmin": 0.001, "steps": 1000, "updates": 100})

    # Run the annealing optimization
    best_tail_ref_pt_config, best_avg_outcome = annealer.anneal()

    if -best_avg_outcome > best_overall_avg_outcome:
        best_overall_tail_ref_pt_config = best_tail_ref_pt_config
        best_overall_avg_outcome = -best_avg_outcome

    print(
        f"Step {i+1}/{n_repeats_SA}: Best tail reference point configuration: {best_tail_ref_pt_config}"
    )
    print(
        f"Step {i+1}/{n_repeats_SA}: Best average outcome: {-best_avg_outcome}"
    )  # Negate to get the maximized outcome

print("Best overall tail reference point configuration:")
pprint(best_overall_tail_ref_pt_config)
print("Best overall average outcome:", best_overall_avg_outcome)

### Prediction with objectionable components decoupled

In [ ]:
# "married": married is coded 6 or 7
tail_ref_pt_config = {
    "male->high_income": 0,
    # "married->higher_edu": 7,
    # "married->managerial_occ": 1,
    "married->high_income": 7,
}

test_whole = DataLoader(testing_dataset, batch_size=len(test_indices), shuffle=False)

batch_test_whole = next(iter(test_whole))
batch_test_whole = {
    node: node_tensor.to(device) for node, node_tensor in batch_test_whole.items()
}

batch_test_whole_output_inference = best_model.inference(
    batch_test_whole,
    feature_standard_scaler_dict,
    tail_ref_pt_config=tail_ref_pt_config,
    tail_ref_pt_config_as_dummy=False,
)

Yhat_ref_pt = batch_test_whole_output_inference["high_income"].numpy()
Yhat_ref_pt = (
    (Yhat_ref_pt > 0.5)
    .astype(int)
    .reshape(
        -1,
    )
)

print("Done. Outputs with CustomNetwork.inference() with reference points.")